In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from scipy import stats
import pydotplus
import graphviz
import os

path = r'C:\Users\owner\Anaconda3\Library\bin\graphviz'
os.environ["PATH"] += os.pathsep + path

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import time

In [2]:
df = pd.read_csv(r'C:\Users\owner\MELBOURNE_HOUSE_PRICES_LESS.csv')

In [3]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [4]:
categorical = df.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

Suburb
375
Address
48396
Type
3
Method
9
SellerG
434
Date
91
Regionname
8
CouncilArea
33


In [5]:
df.drop(['Address', 'SellerG', 'Date'], 1, inplace=True)

In [6]:
df.head()

,Suburb,Rooms,Type,Price,Method,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,3,h,1490000.0,S,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,3,h,1220000.0,S,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,3,h,1420000.0,S,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,3,h,1515000.0,S,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,2,h,670000.0,S,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [7]:
df.shape

(52964, 9)

In [8]:
#appr.22% of the price is missing
df[df.Price.isnull()==True]

,Suburb,Rooms,Type,Price,Method,Regionname,Propertycount,Distance,CouncilArea
8,Albanvale,6,h,NaN,PI,Western Metropolitan,1899,14.0,Brimbank City Council
15,Altona Meadows,4,h,NaN,SN,Western Metropolitan,7630,15.5,Hobsons Bay City Council
24,Ashburton,4,h,NaN,S,Southern Metropolitan,3052,10.2,Boroondara City Council
29,Attwood,4,h,NaN,SN,Northern Metropolitan,1130,16.5,Hume City Council
38,Balwyn North,3,h,NaN,SP,Southern Metropolitan,7809,9.7,Boroondara City Council
42,Balwyn North,4,h,NaN,SP,Southern Metropolitan,7809,9.7,Boroondara City Council
48,Beaumaris,2,t,NaN,VB,Southern Metropolitan,5366,17.3,Bayside City Council
54,Bentleigh,2,h,NaN,S,Southern Metropolitan,6795,11.4,Glen Eira City Council
56,Bentleigh East,2,u,NaN,S,Southern Metropolitan,10969,13.8,Glen Eira City Council
62,Black Rock,3,t,NaN,S,Southern Metropolitan,2866,17.3,Bayside City Council


In [9]:
#dropping all rows where price is NAN

df = df.dropna(subset=['Price'])

In [10]:
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=7
)

X=df[['Rooms', 'Type', 'Price', 'Propertycount', 'Distance']]
X = pd.get_dummies(X)
X =X.dropna(axis=1)
Y=df['Regionname']

decision_tree.fit(X, Y)
print(cross_val_score(decision_tree, X, Y, cv=5).mean())
%timeit pass

0.561412937215201
9.33 ns ± 0.0702 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


# Decision Tree

In [11]:
from sklearn.model_selection import cross_val_score
numbers =[]
scores = []

for number in range(1, 15):
    mod_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth= number
    )
    numbers.append(number)
    scores.append(cross_val_score(mod_tree, X, Y, cv=5).mean())

In [12]:
results = pd.DataFrame(numbers)
results.columns = ['max_depth']
results['scores'] = scores 

In [13]:
results

,max_depth,scores
0,1,0.326946
1,2,0.352554
2,3,0.376201
3,4,0.441428
4,5,0.469729
5,6,0.555224
6,7,0.541657
7,8,0.574083
8,9,0.665215
9,10,0.725557


# Random Forest

In [14]:
from sklearn import ensemble
rfc = ensemble.RandomForestClassifier()

In [15]:
print(cross_val_score(rfc, X, Y, cv=5).mean())

%timeit pass

0.9751920493973495
9.54 ns ± 0.199 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
